# 📈 Project: NVDA Direction & Volatility Prediction

In this project, we predict both:
- Whether NVDA stock will go **UP or DOWN** tomorrow (Direction)
- Whether volatility will be **High or Low** (Risk)

This helps with **strategy planning**: safe vs risky trades.

In [ ]:
!pip install yfinance ta scikit-learn matplotlib --quiet

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error
import ta

In [ ]:
# Load NVDA data
end_date = datetime.today()
start_date = end_date - timedelta(days=365*2)
df = yf.download("NVDA", start=start_date, end=end_date)

# Add features
df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
df['Target'] = (df['LogReturn'].shift(-1) > 0).astype(int)  # 1 if price goes up next day
df['Volatility'] = df['LogReturn'].rolling(window=5).std() * np.sqrt(252)
df['VolatilityLabel'] = (df['Volatility'] > df['Volatility'].mean()).astype(int)

df['Momentum'] = df['Close'] - df['Close'].shift(5)
df['SMA_5'] = df['Close'].rolling(window=5).mean()
df['SMA_10'] = df['Close'].rolling(window=10).mean()
df['RSI'] = ta.momentum.RSIIndicator(close=df[['Close']].squeeze(), window=14).rsi()

df.dropna(inplace=True)

In [ ]:
# Set features and targets
features = ['Momentum', 'SMA_5', 'SMA_10', 'RSI']

# Direction model
X_dir = df[features]
y_dir = df['Target']
X_train_dir, X_test_dir, y_train_dir, y_test_dir = train_test_split(X_dir, y_dir, test_size=0.2, shuffle=False)
clf = LogisticRegression()
clf.fit(X_train_dir, y_train_dir)

# Volatility model
X_vol = df[features]
y_vol = df['VolatilityLabel']
X_train_vol, X_test_vol, y_train_vol, y_test_vol = train_test_split(X_vol, y_vol, test_size=0.2, shuffle=False)
rf_vol = RandomForestClassifier(n_estimators=100)
rf_vol.fit(X_train_vol, y_train_vol)

In [ ]:
# Predict today
latest = df[features].iloc[-1].values.reshape(1, -1)
pred_dir = clf.predict(latest)[0]
pred_vol = rf_vol.predict(latest)[0]

print("🔮 Prediction for Tomorrow:")
print("📈 Direction:", "UP" if pred_dir == 1 else "DOWN")
print("🌪️ Volatility:", "High" if pred_vol == 1 else "Low")

## 🧠 Strategy Guide

| Direction | Volatility | Suggested Strategy |
|-----------|------------|---------------------|
| UP        | Low        | Buy shares, sell puts |
| UP        | High       | Buy call, long straddle |
| DOWN      | Low        | Stay out, covered calls |
| DOWN      | High       | Buy puts, short stock |

In [ ]:
# 📊 Plot actual vs predicted direction and volatility
fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Direction
y_pred_dir_all = clf.predict(X_test_dir)
axs[0].plot(y_test_dir.index, y_test_dir, label='Actual Direction', color='black')
axs[0].plot(y_test_dir.index, y_pred_dir_all, label='Predicted Direction', linestyle='--', color='blue')
axs[0].set_title('Direction: Actual vs Predicted')
axs[0].legend()
axs[0].grid(True)

# Volatility
y_pred_vol_all = rf_vol.predict(X_test_vol)
axs[1].plot(y_test_vol.index, y_test_vol, label='Actual Volatility', color='black')
axs[1].plot(y_test_vol.index, y_pred_vol_all, label='Predicted Volatility', linestyle='--', color='green')
axs[1].set_title('Volatility: Actual vs Predicted')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.xticks(rotation=45)
plt.show()